Try timestamp input with time HH:MM:SS

! timedelta(days=1) = timedelta(hours=24) -> doesn't change at midnight/partial days

Would do with more time:

Would find way to calculate calendar days vs. 24hr gaps to filter by TIME for any accounts repeating in a day -- OR -- would select later time (if timestamp occurs >1, select latest)

Would fix input to find date even if input doesn't match format

Find nearest date if date doesn't occur: if timestamp_dt is between date 1 and date 2, return to_state = date 1

In [23]:
import pandas as pd
from datetime import timedelta

In [25]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast_df = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [ ]:
ast_df.head()

In [ ]:
len(ast_df.account_id.unique())

## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

### Limit Dataframe by Indicated Timestamp

In [ ]:
# for all ACCOUNT_ID, return TO_STATE where STARTED_WHEN = TIMESTAMP OR nearest_preceding_date
# if ACCOUNT_ID doesn't have corresponding TIMESTAMP select earliest preceding date with record (preceding_date)
# select TO_STATE of PRECEDING_DATE and calculate time difference: TIMESTAMP-PRECEDING_DATE

In [24]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [26]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [27]:
# Change data type from string to datetime object
ast_df['started_when'] = pd.to_datetime(ast_df['started_when'])

In [28]:
# Create limited dataframe based on indicated timestamp 
df_limited = ast_df.loc[ast_df['started_when'] <= (timestamp_dt + timedelta(hours=23,minutes=59,seconds=59))]
df_limited.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [32]:
# df_limited['started_when'].max()
# MIN 2019-04-25 01:24:38.571392
# MAX 2020-03-01 23:52:15.571392

Timestamp('2020-03-01 23:52:15.571392')

### Current State of Each Account by Indicated Timestamp

In [ ]:
# Create dataframe to show current state of each account based on indicated timestamp
df_limited2 = df_limited[['started_when','account_id','to_state']]

# Sort by account_id
df_limited2 = df_limited2.sort_values('account_id')

# Reset index
df_limited2 = df_limited2.reset_index(drop=True)
df_limited2.head()

In [ ]:
# Show in dataframe, any accounts occuring more than once during indicated timestamp

account_ids = []
df_repeat = pd.DataFrame(columns=['started_when', 'account_id', 'to_state'])

for i in range(0,len(df_limited2['account_id'])):
    account_id = df_limited2['account_id'][i]
    if account_id in account_ids:
        df_temp = df_limited2.loc[df_limited2['account_id'] == account_id]
        df_repeat = df_repeat.append(df_temp)
    else:
        account_ids.append(account_id)

df_repeat

### Total Time in State as of Indicated Timestamp

In [ ]:
# if total time = start to timestamp:
# for each account id, select state = current state for all records <= timestamp
